# Predicting Career Satisfaction From Stack Overflow Data


## Read the dataset


In [74]:
import pandas as pd
dataset = pd.read_csv('./survey_results_public.csv')


C:\Users\lewys\Miniconda3\envs\mlfinal\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (8,12,13,14,15,16,50,51,52,53,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Separate career satisfaction data from the rest of the columns


In [75]:
# remove all rows from the dataset where career satisfaction has no data
dataset = dataset[dataset['CareerSatisfaction'].notnull()]
# load the career satisfaction column into y
y = dataset['CareerSatisfaction']
# load the the dataset minus the career satisfaction column into x
print(dataset.shape)
X = dataset.drop('CareerSatisfaction', axis=1)
print(X.shape)


(76504, 129)
(76504, 128)


## Create dummy variables for y


In [76]:
print(y.shape)
y = pd.get_dummies(y)
# notice the shape after creating dummies is (76504, 7), because get_dummies() created a new column for each category of answer in the survey
print(y.shape)


(76504,)
(76504, 7)


## Clean the rest of the dataset


### Choose a subset of variables from the dataset for training data


In [77]:
X_subset = X[[
               'LanguageWorkedWith',
               'DatabaseWorkedWith',
               'PlatformWorkedWith',
               'FrameworkWorkedWith',
               'OperatingSystem',
               'NumberMonitors',
               'StackOverflowHasAccount',
               'HoursComputer',
               'HoursOutside'
]]

print('shape before dropna: ', X_subset.shape)
# drop any rows with null data
X_subset = X_subset.dropna()
print('shape after dropna: ', X_subset.shape)


shape before dropna:  (76504, 9)
shape after dropna:  (36581, 9)


### For the `FrameworkWorkedWith` column, one-hot encode each answer

The responses in the `FrameworkWorkedWith` column correspond to a semicolon separated list (e.g `.NET Core;Spark`).
The goal of this section of the code is twofold:
1. find all the Frameworks which respondents were able to choose from
2. turn each string response into a one-hot encoded vector (e.g. `.NET Core;Spark` becomes `[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]`)



In [78]:
def one_hot_encode(categories, data):
    '''
    takes in a list of categories and a list of data and returns a one-hot encoded vector 
    :param categories: 
    :param data: 
    :return: 
    '''
    one_hot_vector = [0 for _ in range(len(categories))] # initialize a vector with 0s for each category
    for item in data:
        for i, category in enumerate(categories):
            if item == category:
                one_hot_vector[i] = 1
    return one_hot_vector


In [79]:
# start by getting every possible category of response
set_of_responses = set()
for resp in X_subset['FrameworkWorkedWith']:
    # parse the response into a list of values
    resp_list = resp.split(';')
    # replace the response in the dataframe with the parsed list of values
    
    # add to the set of total responses
    resp_set = set(resp_list)
    set_of_responses.update(resp_set)
# put the frameworks in alphabetical order for prettiness
sorted_set_of_responses = sorted(set_of_responses)


Now that we've parsed all the data and found all the categories of responses for the FrameworkWorkedWith column, we can one-hot encode it easily


In [80]:
# iterate thru responses once more to map the list of responses into one-hot encoded columns
data = {}
for i, resp in X_subset['FrameworkWorkedWith'].iteritems():
    resp_list = resp.split(';')
    one_hot_vector = one_hot_encode(list(sorted_set_of_responses) , resp_list)
    data[i] = one_hot_vector

# build a new dataframe that will replace the FrameworkWorkedWith column with the dict we just built
framework_col_replacement = pd.DataFrame().from_dict(data, orient='index', columns=list(sorted_set_of_responses))
print(framework_col_replacement.head())


   .NET Core  Angular  Cordova  Django  Hadoop  Node.js  React  Spark  Spring  \
0          0        0        0       1       0        0      1      0       0   
1          0        0        0       1       0        0      0      0       0   
5          0        1        0       0       0        1      0      0       0   
6          0        0        0       0       0        1      1      0       0   
7          0        1        0       0       0        1      0      0       0   

   TensorFlow  Torch/PyTorch  Xamarin  
0           0              0        0  
1           0              0        0  
5           0              0        0  
6           0              0        0  
7           0              0        0  


### Now replace the FrameworkWorkedWith column with the new one hot encoded df we just built


In [81]:
X_subset_no_framework_col = X_subset.drop('FrameworkWorkedWith', axis=1)
print('before framework df', X_subset_no_framework_col.shape)
print(X_subset_no_framework_col.head())
X_subset_one_hot = pd.concat([X_subset_no_framework_col, framework_col_replacement], axis=1)
print('with new framework df: ', X_subset_one_hot.shape)
print(X_subset_one_hot.head())


before framework df (36581, 8)
                           LanguageWorkedWith  \
0                  JavaScript;Python;HTML;CSS   
1                JavaScript;Python;Bash/Shell   
5  Java;JavaScript;Python;TypeScript;HTML;CSS   
6                         JavaScript;HTML;CSS   
7              JavaScript;TypeScript;HTML;CSS   

                                  DatabaseWorkedWith  \
0  Redis;SQL Server;MySQL;PostgreSQL;Amazon RDS/A...   
1                         Redis;PostgreSQL;Memcached   
5                                            MongoDB   
6                                            MongoDB   
7  MongoDB;MySQL;Microsoft Azure (Tables, CosmosD...   

         PlatformWorkedWith OperatingSystem NumberMonitors  \
0  AWS;Azure;Linux;Firebase     Linux-based              1   
1                     Linux     Linux-based              2   
5                     Linux     Linux-based              2   
6                     Linux           MacOS              2   
7              Azure;Heroku

## Actually fit a model now that the data is clean


In [82]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split

In [83]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
# print(X.sum(axis=1))
X_smol = X[:500]
print(X_smol.shape)
print(X_smol.columns[X_smol.sum() == 1])
# print(X.head())

In [ ]:
sel = SelectFromModel(RandomForestClassifier(n_estimators=100))
sel.fit(X_train, y_train)